# Exercise_1

Basic idea: Choose scenario with click on a button in the GUI. Then click on a Start button to run the simulation. \
    In each timestep of the simulation the new (precise) position of each pedestrian is calculated. Afterwards the discretized position in the grid/cellular automaton is shown in the GUI.

In [1]:
import tkinter as tk
import numpy as np
import math
import time
import random
#import pygame #could be an alternative to tkinter

In [2]:
class Person():
    '''
    pos_x, pos_y: precise position of pedestrian (Idea: cells are discretized positions we only use for drawing)
    speed: movement speed of pedestrian in m/s
    move(): see below
    '''
    def __init__(self, position_x, position_y, speed=1):
        self.pos_x = position_x
        self.pos_y = position_y
        self.speed = speed

        
    def move(self, grid, scenario):
        '''
        equals an update for the position depending on speed, interaction of pedestrians and obstacle avoidance
        costField is discritized per se, so position needs to be discretized as well
        looks for minimal cost in all directions within the feasible area
        '''
        self.pedestrianInteraction(scenario)#updates interactionField
        costField = np.array(scenario.costField) + np.array(self.interactionField) #+ obstacleAvoidance
        (x_opt, y_opt) = (self.pos_x, self.pos_y)
        cost_min = scenario.costField[int(self.pos_x)][int(self.pos_y)]

        #quick and dirty part
        
        dxplus = self.pos_x + self.speed
        dxminus = self.pos_x - self.speed
        dyplus = self.pos_y + self.speed
        dyminus = self.pos_y - self.speed
        
        dxx = (self.pos_x, dxplus, dxminus)
        dyy = (self.pos_y, dyplus, dyminus)
        
        cell_move = (math.sqrt(scenario.rows**2 + scenario.columns**2), self.pos_x, self.pos_y)             
        
        for x in dxx:
            for y in dyy:
                if grid.grid[x][y] !='T':
                    if grid.grid[x][y] !='P' and (x,y) != (self.pos_x, self.pos_y):
                        cell_dist = scenario.costField[x][y]
                        if cell_move[0] > cell_dist:
                            cell_move = (cell_dist,x,y)
                    

        grid.grid[int(self.pos_x)][int(self.pos_y)]='E'
        grid.grid[int(cell_move[1])][int(cell_move[2])]='P'
        self.pos_x = cell_move[1]
        self.pos_y = cell_move[2]
        
        
    def pedestrianInteraction(self, scenario):
        self.interactionField = [[0.0 for y in range(scenario.columns)] for x in range(scenario.rows)]
        for p in scenario.persons:
            if p == self:
                continue#this is to avoid penalty with yourself. I assume you like yourself :)
            else:
                x = int(p.pos_x)
                y = int(p.pos_y)
                self.interactionField[x][y] = math.exp(1/(scenario.rmax**2))
                for r in range(scenario.rmax):
                    #TODO: diagonal be careful
                    if x-r >= 0:
                        self.interactionField[x-r][y] += math.exp(-1/(r**2 - scenario.rmax**2))
                    if y+r < scenario.columns:
                        self.interactionField[x][y+r] += math.exp(-1/(r**2 - scenario.rmax**2))
                    if x+r < scenario.rows:
                        self.interactionField[x+r][y] += math.exp(-1/(r**2 - scenario.rmax**2))
                    if y-r >= 0:
                        self.interactionField[x][y-r] += math.exp(-1/(r**2 - scenario.rmax**2))

In [3]:
class Scenario():
    '''
    creates persons, targets, obstacles and grid size depending on the scenario
    '''
    def __init__(self):
        self.rows = 25
        self.columns = 25
        self.persons = []
        self.targets = []
        self.obstacles = []
        self.grid = Grid(self)
        self.rmax = 1 # constant value at the moment
        #self.setTo1()
        
    def setTo1(self):#one person straight
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 25)]
        self.targets = [(25, 25)]
        self.obstacles = []
        self.setupCostField()
        self.grid = Grid(self)
        
    def setTo2(self):#five persons in a circle
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 25), Person(25, 35), Person(25, 15), Person(40, 30), Person(40, 20)]
        self.targets = [(25, 25)]
        self.obstacles = []
        self.setupCostField()
        self.grid = Grid(self)
        
    def setTo3(self):#bottleneck
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 25), Person(40, 20), Person(34, 15), Person(15,23), Person(25,5), Person(46,17), Person(22,45)]
        self.targets = [(26, 26)]
        self.obstacles = []
        self.setupCostField()
        self.grid = Grid(self)
        
    def setTo4(self):#chickentest
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 5), Person(10, 7), Person(3, 2), Person(18, 3), Person(46, 44), Person(38,41), Person(45,24), Person(45,2)]
        self.targets = [(27, 27)]
        self.obstacles = []
        self.setupCostField()
        self.grid = Grid(self)

    def setTo5(self):#added 5th scenario to test hypothesis
        self.rows = 50
        self.columns = 50
        self.persons = [Person(5, 45), Person(10, 47), Person(3, 42), Person(18, 43), Person(46, 44), Person(38,41), Person(45,44), Person(45,42)]
        self.targets = [(25, 0)]
        self.obstacles = []
        self.setupCostField()
        self.grid = Grid(self)
        
    #TODO: addd RiMEA tests
    #TODO: change persons, obstacles and maybe targets for scenarios 3 and 4
        
    def setupCostField(self):
        #cost scaled by max possible cost/distance
        self.costField = [[1.0 for y in range(self.columns)] for x in range(self.rows)]
        max_dist = math.sqrt(self.rows**2 + self.columns**2)
        for x in range(self.rows):
            for y in range(self.columns):
                if (x,y) in self.targets:
                    self.costField[x][y] = 0
                else:#computes cost/distance for nearest target
                    dist = max_dist
                    for t in self.targets:
                        min_dist = math.sqrt((x - t[0])**2 + (y - t[1])**2)
                        if min_dist < dist:
                            dist = min_dist
                    cost = dist/max_dist
                    self.costField[x][y] = cost
                    
    def move(self):
        for p in self.persons:
            p.move(self.grid, self)

In [4]:
class Grid():
    '''
    Basically only for visualization.
    '''
    def __init__(self, scenario):
        self.scenario = scenario
        self.grid = [['E' for y in range(scenario.columns)] for x in range(scenario.rows)]
        for p in scenario.persons:
            self.grid[p.pos_x][p.pos_y] = 'P'
        for t in scenario.targets:
            self.grid[t[0]][t[1]] = 'T'
        for o in scenario.obstacles:
            self.grid[o[0]][o[1]] = 'O'
        
    def draw(self, canvas, line_distance):
        '''
        Sould draw the grid and the persons/targets/obstacles.
        Depends on tkinter stuff. I have to look at this in more detail.
        '''
        # vertical lines
        for x in range(line_distance,canvas.winfo_width(),line_distance):
            canvas.create_line(x, 0, x, canvas.winfo_height(), fill="#476042")
        # horizontal lines
        for y in range(line_distance,canvas.winfo_height(),line_distance):
            canvas.create_line(0, y, canvas.winfo_width(), y, fill="#476042")
        # P, T, O or nothing in between the lines
        for (x, y), value in np.ndenumerate(self.grid):
            if value == 'P':
                color = 'green'
            elif value == 'T':
                color = 'red'
            elif value == 'O':
                color = 'black'
            else:
                value = ''
                color = 'white'
            canvas.create_text((x+0.5) * line_distance, (y+0.5) * line_distance, text=value, fill=color)
            #Rectangle Vizualisation seems to cause some lag...
            #canvas.create_rectangle(x*line_distance, y*line_distance, (x+1)*line_distance,(y+1)*line_distance, fill=color)
            #canvas.create_rectangle(..) as possible improvement for visualization

In [5]:
def main():
    '''
    Setup of the interface stuff.
    When a scenario is selected via a button, it should appear next to the buttons in the canvas.
    When clicking on start button, the current scenario should be simulated, i.e. for every timestep=1s each pedestrian should move() and the drawing should be renewed.
    
    '''
    window = tk.Tk()
    window.title('Exercise 1')
    
    ###->
    scenario = Scenario()
    
    #I'm not sure, whether a canvas is the best option, perhaps a frame is better.
    ###->
    #TODO: make line_distance dependend on scenario columns/rows for nicer layout. Same for fontsize?
    line_distance = 20#should be passed to draw() in Grid
    canvas_width = line_distance * scenario.columns
    canvas_height = line_distance * scenario.rows
    canvas = tk.Canvas(master=window, width=canvas_width, height=canvas_height)
    canvas.pack(side='left')
    
    btnScenario1 = tk.Button(master=window, text='Scenario 1', bg='#26f9ad', width='8', command=lambda: scenario.setTo1())
    btnScenario1.pack()#(side=RIGHT)
    btnScenario2 = tk.Button(master=window, text='Scenario 2', bg='#26f9ad', width='8', command=lambda: scenario.setTo2())
    btnScenario2.pack()#(side=RIGHT)
    btnScenario3 = tk.Button(master=window, text='Scenario 3', bg='#26f9ad', width='8', command=scenario.setTo3)
    btnScenario3.pack()#(side=RIGHT)
    btnScenario4 = tk.Button(master=window, text='Scenario 4', bg='#26f9ad', width='8', command=scenario.setTo4)
    btnScenario4.pack()#(side=RIGHT)
    btnScenario5 = tk.Button(master=window, text='Scenario 5', bg='#26f9ad', width='8', command=scenario.setTo5)
    btnScenario5.pack()#(side=RIGHT)
    
    #TODO: add buttons for RiMEA tests
    
    ####->
    def start(window, canvas, scenario, time_limit=100):
        canvas_width = line_distance * scenario.columns
        canvas_height = line_distance * scenario.rows
        canvas.configure(width=canvas_width, height=canvas_height)
        timesteps = 0
        while timesteps < time_limit:
            scenario.move()
            canvas.delete('all')
            time.sleep(0.2)
            window.after(200, lambda: scenario.grid.draw(canvas, line_distance))
            window.update()
            timesteps += 1
    
    btnStart = tk.Button(master=window, text="Start", command=lambda: start(window, canvas, scenario))
    btnStart.pack()
    
    #Below is an attempt to display explanations for the scenrio buttons. As this is rather cosmetic I suggest to do it at the end.
    #explanation = tk.Text(master=window, height=4, state=DISABLED)
    #explanation.insert(INSERT, "First scenario is one pedestrain walking on a straight line to the target.\n Second scenario is five pedestrians in a circle around the target.\n Third scenario is bottleneck.\n Fourth scenario is chicken test.\n")
    #explanation.pack(side=BOTTOM)
    
    window.mainloop()

In [6]:
if __name__ == "__main__":
    main()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\golde\Anaconda3\envs\MLDL\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-5-6c23e11da1e9>", line 50, in <lambda>
    btnStart = tk.Button(master=window, text="Start", command=lambda: start(window, canvas, scenario))
  File "<ipython-input-5-6c23e11da1e9>", line 44, in start
    canvas.delete('all')
  File "C:\Users\golde\Anaconda3\envs\MLDL\lib\tkinter\__init__.py", line 2514, in delete
    self.tk.call((self._w, 'delete') + args)
_tkinter.TclError: invalid command name ".!canvas"
